In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# load dataset
df = pd.read_csv('../data/pima-indians-diabetes.csv')
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-hour serum insulin (μIU/ml),Body mass index (weight in kg/(height in m)2),Diabetes pedigree function,Age (years),Diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X_np = df.iloc[:, 0:8].values
y_np = df.iloc[:, 8].values

X_np.shape, y_np.shape

((768, 8), (768,))

In [4]:
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).reshape(-1, 1)

X.shape, y.shape, X.type, y.type

(torch.Size([768, 8]),
 torch.Size([768, 1]),
 <function Tensor.type>,
 <function Tensor.type>)

In [5]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x


model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [6]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        
        # forward propagation
        X_batch = X[i : i + batch_size]
        y_pred = model(X_batch)
        y_batch = y[i : i + batch_size]
        loss = loss_fn(y_pred, y_batch)
        
        # back propagation
        optimizer.zero_grad() # make previous gradients zero, else new gradient will be added to them.
        loss.backward() # start computing gradient and back propagate
        optimizer.step() # apply the computed gradients
        
    print(f"Finished epoch {epoch+1}, latest loss {loss}")

Finished epoch 1, latest loss 0.6816895008087158
Finished epoch 2, latest loss 0.6228927373886108
Finished epoch 3, latest loss 0.610573410987854
Finished epoch 4, latest loss 0.5981374979019165
Finished epoch 5, latest loss 0.5805274248123169
Finished epoch 6, latest loss 0.5745772123336792
Finished epoch 7, latest loss 0.5690314769744873
Finished epoch 8, latest loss 0.5636220574378967
Finished epoch 9, latest loss 0.5591604113578796
Finished epoch 10, latest loss 0.5568035244941711
Finished epoch 11, latest loss 0.5488308072090149
Finished epoch 12, latest loss 0.5326057076454163
Finished epoch 13, latest loss 0.5191224217414856
Finished epoch 14, latest loss 0.5008581876754761
Finished epoch 15, latest loss 0.4625738561153412
Finished epoch 16, latest loss 0.43183189630508423
Finished epoch 17, latest loss 0.42300736904144287
Finished epoch 18, latest loss 0.4158216416835785
Finished epoch 19, latest loss 0.4073503315448761
Finished epoch 20, latest loss 0.39654168486595154
Finishe

In [7]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)

accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.76953125
